In [32]:
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from tensorflow.keras.models import load_model
import numpy as np
from pydub import AudioSegment
import tempfile
import soundfile as sf
import librosa
import os


In [33]:
# Step 1: Load and Preprocess Data

lyrics_folder = "Lyrics"
audio_folder = "Audio"

lyrics = []
audio_features = []

# Load lyrics from text files
for filename in os.listdir(lyrics_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(lyrics_folder, filename), "r", encoding="utf-8") as file:
            lyrics.append(file.read())

# Preprocess lyrics
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in lyrics:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Process audio files
def preprocess_audio(audio_path):
    y, sr = librosa.load(audio_path)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    return mel_spectrogram

for filename in os.listdir(audio_folder):
    if filename.endswith(".mp3"):
        audio_path = os.path.join(audio_folder, filename)
        audio_feature = preprocess_audio(audio_path)
        audio_features.append(audio_feature)

In [34]:
# Save processed audio features
if not os.path.exists("processed_audio"):
    os.makedirs("processed_audio")

for i, audio_feature in enumerate(audio_features):
    output_path = os.path.join("processed_audio", f"audio_{i}.npy")
    np.save(output_path, audio_feature)

# Save tokenizer for lyrics
with open("lyrics_tokenizer.pkl", "wb") as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

# Save input sequences for lyrics generation
np.save("input_sequences.npy", input_sequences)

print("Step 1: Data loading and preprocessing completed.")

Step 1: Data loading and preprocessing completed.


In [35]:
# Load preprocessed data
input_sequences = np.load("input_sequences.npy")
tokenizer = None

# Load the saved tokenizer
with open("lyrics_tokenizer.pkl", "rb") as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Tokenize lyrics
total_words = len(tokenizer.word_index) + 1
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [36]:
# Define the RNN model
model = Sequential()
model.add(Embedding(total_words, 64, input_length=input_sequences.shape[1]-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Save the trained model
model.save("lyrics_generation_model.h5")

print("Step 2: Model building and training completed.")

Epoch 1/100
55/55 [==============================] - 9s 132ms/step - loss: 6.3424 - accuracy: 0.0289
Epoch 2/100
55/55 [==============================] - 7s 135ms/step - loss: 6.0584 - accuracy: 0.0277
Epoch 3/100
55/55 [==============================] - 8s 143ms/step - loss: 5.8362 - accuracy: 0.0301
Epoch 4/100
55/55 [==============================] - 8s 144ms/step - loss: 5.6519 - accuracy: 0.0260
Epoch 5/100
55/55 [==============================] - 8s 144ms/step - loss: 5.4320 - accuracy: 0.0382
Epoch 6/100
55/55 [==============================] - 8s 145ms/step - loss: 5.1243 - accuracy: 0.0607
Epoch 7/100
55/55 [==============================] - 8s 148ms/step - loss: 4.8251 - accuracy: 0.0798
Epoch 8/100
55/55 [==============================] - 8s 151ms/step - loss: 4.5146 - accuracy: 0.1017
Epoch 9/100
55/55 [==============================] - 8s 148ms/step - loss: 4.2312 - accuracy: 0.1202
Epoch 10/100
55/55 [==============================] - 8s 155ms/step - loss: 3.9606 - accura

c:\Users\MichaelKim\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
# Load the saved tokenizer
with open("lyrics_tokenizer.pkl", "rb") as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Load the trained model
model = load_model("lyrics_generation_model.h5")

In [48]:
# Generate Lyrics

seed_text = "Cupid"
next_words = 150

generated_lyrics = []

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=input_sequences.shape[1]-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
    generated_lyrics.append(output_word)

# Combine generated lyrics into lines
formatted_lyrics = "\n".join(" ".join(generated_lyrics[i:i+8]) for i in range(0, len(generated_lyrics), 8))

# Print the formatted lyrics
print("Generated Lyrics:")
print(formatted_lyrics)

# Write the formatted lyrics to a text file
output_lyrics_path = "formatted_lyrics.txt"
with open(output_lyrics_path, "w", encoding="utf-8") as output_file:
    output_file.write(formatted_lyrics)

print(f"Formatted lyrics saved to '{output_lyrics_path}'.")

1/1 [==============================] - 0s 26ms/step
Generated Lyrics:
are you happy now finally happy now are
you 뭐 그대로야 난 다 잃어버린 것 같아
모든 게 맘대로 왔다가 인사도 없이 떠나 이대로는
무엇도 사랑하고 싶지 않아 다 해질 대로 해져버린
기억 속을 여행해 우리는 오렌지 태양 아래 그림자
없이 함께 춤을 춰 정해진 이별 따위는 없어
아름다웠던 그 기억에서 만나 forever young 우우우 우우우우
우우우 우우우우 forever we young 우우우 우우우우 이런
악몽이라면 영영 깨지 않을게 섬 그래 여긴 섬
서로가 만든 작은 섬 예 음 forever young
영원이란 말은 모래성 작별은 마치 재난문자 같지 그리움과
같이 맞이하는 아침 서로가 이 영겁을 지나 꼭
이 섬에서 다시 만나 지나듯 날 위로하던 누구의
말대로 고작 한 뼘짜리 추억을 잊는 게 참
쉽지 않아 시간이 지나도 여전히 날 붙드는 그곳에
우리는 오렌지 태양 아래 그림자 없이 함께 춤을
춰 정해진 안녕 따위는 없어 아름다웠던 그 기억에서
만나 우리는 서로를 베고 누워 슬프지 않은 이야기를
나눠 우울한 결말 따위는 없어 난
Formatted lyrics saved to 'formatted_lyrics.txt'.


In [9]:
# Step 4: Generate Audio and Combine
def overlay_lyrics_on_audio(lyrics, audio_features):
    combined_audio = np.zeros_like(audio_features[0])

    # Simulate synchronization and alignment of lyrics with audio
    for i, audio_segment in enumerate(audio_features):
        start_index = i * len(audio_segment)
        end_index = (i + 1) * len(audio_segment)

        # Apply lyrics to the audio segment
        combined_audio[start_index:end_index] += audio_segment

    return combined_audio

In [10]:
# Overlay generated lyrics on generated audio features
generated_audio_features = [np.load("processed_audio/audio_0.npy")]  # Load your generated audio features
combined_audio = overlay_lyrics_on_audio(generated_lyrics, generated_audio_features)
print(combined_audio)

[[1.29536030e-41 1.48825206e-27 4.97319386e-15 ... 6.61177751e-07
  5.63870742e-07 1.87379428e-08]
 [1.59117441e-41 1.50518628e-27 1.28851008e-14 ... 2.09218982e-07
  8.44328767e-08 7.83840015e-09]
 [1.72107477e-41 1.45666344e-27 1.82865536e-14 ... 1.15953526e-07
  4.11922763e-08 2.95455149e-09]
 ...
 [8.18358303e-42 9.29594565e-29 1.40628825e-15 ... 1.38811596e-09
  1.06636214e-10 2.53857509e-13]
 [1.01636178e-41 1.20817160e-28 9.80406543e-16 ... 3.43799900e-09
  1.53560539e-10 1.57561708e-13]
 [1.36248250e-41 7.29179707e-29 1.36965906e-17 ... 5.68306791e-10
  1.38731448e-11 3.18923194e-15]]


In [12]:
from pydub import AudioSegment

In [14]:
target_sample_rate = 44100  # Adjust as needed


In [15]:
# Convert the combined audio to an AudioSegment object
combined_audio = AudioSegment(combined_audio.tobytes(), frame_rate=target_sample_rate, sample_width=combined_audio.dtype.itemsize, channels=1)  # Adjust the number of channels as needed

# Export the combined audio as a WAV file
output_audio_path = "output_cover_song.wav"  # Adjust the output path
combined_audio.export(output_audio_path, format="wav")

print("Step 4: Audio generation and combination completed.")

Step 4: Audio generation and combination completed.


In [41]:
print(generated_lyrics)
print(len(generated_lyrics))

you are you happy now finally happy now are you 뭐 그대로야 난 다 잃어버린 것 같아 모든 게 맘대로 왔다가 인사도 없이 떠나 이대로는 무엇도 사랑하고 싶지 않아 다 해질 대로 해져버린 기억 속을 여행해 우리는 오렌지 태양 아래 그림자 없이 함께 춤을 춰 정해진 이별 따위는 없어 아름다웠던 그 기억에서 만나 forever young 우우우 우우우우 우우우 우우우우 forever we young 우우우 우우우우 이런 악몽이라면 영영 깨지 않을게 섬 그래 여긴 섬 서로가 만든 작은 섬 예 음 forever young 영원이란 말은 모래성 작별은 마치 재난문자 같지 그리움과 같이 맞이하는 아침 서로가 이 영겁을 지나 꼭 이 섬에서 다시 만나 지나듯 날 위로하던 누구의 말대로 고작 한 뼘짜리 추억을 잊는 게 참 쉽지 않아 시간이 지나도 여전히 날 붙드는 그곳에 우리는 오렌지 태양 아래 그림자 없이 함께 춤을 춰 정해진 안녕 따위는 없어 아름다웠던 그 기억에서 만나 우리는 서로를 베고 누워 슬프지 않은 이야기를 나눠 우울한 결말 따위는 없어 난 영원히 널 이 기억에서 만나 forever young 우우우 우우우우 우우우 우우우우 forever we young 우우우 우우우우 이런 악몽이라면 영영 깨지 않을게 섬 그래 여긴 섬 서로가 만든 작은 섬 예 음 forever young 영원이란 말은 모래성 작별은 마치 재난문자 같지 그리움과 같이 맞이하는 아침 서로가 이 영겁을 지나 꼭 이 섬에서 다시 만나 지나듯 날 위로하던 누구의 말대로 고작 한 뼘짜리 추억을 잊는 게 참 쉽지 않아 시간이 지나도 여전히 날 붙드는 그곳에 우리는 오렌지 태양 아래 그림자 없이 함께 춤을 춰 정해진 안녕 따위는 없어 아름다웠던 그 기억에서 만나 우리는 서로를 베고 누워 슬프지 않은 이야기를 나눠 우울한 결말 따위는 없어 난 영원히 널 이 기억에서 만나 forever young 우우우 우우우우 우우우 우우우우 forever we young 우우우 우우우우 

In [29]:
import soundfile as sf